In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from tensorflow.keras import Sequential, layers
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", False)
from sklearn.metrics import mean_absolute_percentage_error
from datetime import date, timedelta, datetime
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam
######################################################
from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx import Trainer
from gluonts.evaluation import make_evaluation_predictions
import matplotlib.pyplot as plt

# Prepare dataset

## Load data

In [18]:
df = pd.read_csv('../data/data_preparation.csv', index_col=[0])
df=df.iloc[:-60]

## Fill missing data

In [19]:
def replace_val(df,date1,date2): #on remplace les NaN dans l intervalle de date
    for col in df.columns:
        imputer = KNNImputer(n_neighbors=7) # Instantiate a SimpleImputer object with your strategy of choice
        imputer.fit(df[[col]]) # Call the "fit" method on the object
        df[col] = imputer.transform(df[[col]]) # Call the "transform" method on the object

In [20]:
df['Date']=pd.to_datetime(df['Date'], errors='coerce')
df = df.set_index('Date').asfreq('D')
replace_val(df,'2022-04-30','2022-05-06')

### Select relevant features

In [21]:
df.drop(columns=['t - 1', 't - 2', 't - 3', 't - 4', 't - 5',
       't - 6', 't - 7', 't - 8', 't - 9', 't - 10', 't - 11', 't - 12',
       't - 13', 't - 14', 't - 15', 't - 16', 't - 17', 't - 18', 't - 19',
       't - 20', 't - 21', 't - 22', 't - 23', 't - 24', 't - 25', 't - 26',
       't - 27', 't - 28', 't - 29', 't - 30','sin365_1', 'cos365_1', 'sin365_2', 'cos365_2',
       'sin365_3', 'cos365_3','bank_holiday', 'school_holidays',
       'season','Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)',
       'Hydraulique (MW)', 'Pompage (MW)', 'Bioénergies (MW)',
       'Ech. physiques (MW)', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'TCO Thermique (%)',
       'TCH Thermique (%)', 'TCO Nucléaire (%)', 'TCH Nucléaire (%)',
       'TCO Eolien (%)', 'TCH Eolien (%)', 'TCO Solaire (%)',
       'TCH Solaire (%)', 'Column 30','Code INSEE région','sin_month', 'cos_month',
        'num_day',
        'YEAR', 'MONTH', 'DAY',
        'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE',
       'T2M_MAX', 'T2M_MIN','PS', 'QV2M','WS10M','WS50M','RH2M','week_day' #temoin
        #'PRECTOTCORR' # PRECTOTCORR is bias-corrected total precipitation
        #'sin_day','cos_day'
                ],inplace=True)

In [22]:
df['id']='A'

In [23]:
df

,Consommation (MW),T2M,PRECTOTCORR,sin_day,cos_day,id
Date,,,,,,
2013-02-02,262321.0,5.55,1.94,-0.974928,-0.222521,A
2013-02-03,273743.0,2.96,0.02,-0.781831,0.623490,A
2013-02-04,295369.0,4.67,0.00,0.000000,1.000000,A
2013-02-05,287424.0,6.65,0.19,0.781831,0.623490,A
2013-02-06,294084.0,2.71,0.10,0.974928,-0.222521,A
...,...,...,...,...,...,...
2022-03-07,273204.0,5.98,0.36,0.000000,1.000000,A
2022-03-08,280638.0,5.10,0.83,0.781831,0.623490,A
2022-03-09,270148.0,6.59,0.11,0.974928,-0.222521,A


In [24]:
nb_data_train = int(len(df)*0.85)
nb_data_val = int(len(df)*0.15)

In [25]:
N_test = 14 #on prédit les 14 derniers jours dans chaque dataframe

In [26]:
df_train=df.iloc[:nb_data_train]

In [27]:
df_val = df[nb_data_train:nb_data_train+nb_data_val]

In [28]:
df_train

,Consommation (MW),T2M,PRECTOTCORR,sin_day,cos_day,id
Date,,,,,,
2013-02-02,262321.0,5.55,1.94,-0.974928,-0.222521,A
2013-02-03,273743.0,2.96,0.02,-0.781831,0.623490,A
2013-02-04,295369.0,4.67,0.00,0.000000,1.000000,A
2013-02-05,287424.0,6.65,0.19,0.781831,0.623490,A
2013-02-06,294084.0,2.71,0.10,0.974928,-0.222521,A
...,...,...,...,...,...,...
2020-10-24,193881.0,13.54,0.21,-0.974928,-0.222521,A
2020-10-25,193856.0,14.27,0.31,-0.781831,0.623490,A
2020-10-26,209799.0,12.45,3.84,0.000000,1.000000,A


In [29]:
df_val

,Consommation (MW),T2M,PRECTOTCORR,sin_day,cos_day,id
Date,,,,,,
2020-10-29,209515.0,13.01,0.07,0.433884,-0.900969,A
2020-10-30,205857.0,14.46,0.00,-0.433884,-0.900969,A
2020-10-31,197481.0,13.38,0.10,-0.974928,-0.222521,A
2020-11-01,194049.0,12.63,0.06,-0.781831,0.623490,A
2020-11-02,206305.0,14.18,0.11,0.000000,1.000000,A
...,...,...,...,...,...,...
2022-03-06,261730.0,5.27,1.41,-0.781831,0.623490,A
2022-03-07,273204.0,5.98,0.36,0.000000,1.000000,A
2022-03-08,280638.0,5.10,0.83,0.781831,0.623490,A


In [30]:
#df=df.copy()
#df['Consommation (MW)'].iloc[-N_test:]=0 ###Commenter cette ligne pour preuve du bon fonctionnement

# Build the model

In [31]:
exo_feature = ['T2M','PRECTOTCORR', 'sin_day', 'cos_day']

In [32]:
ds_train = PandasDataset(df_train, target="Consommation (MW)",feat_dynamic_real=exo_feature)
ds_val = PandasDataset(df_val, target="Consommation (MW)",feat_dynamic_real=exo_feature)

In [71]:
estimator = DeepAREstimator(
    freq=ds_train.freq, prediction_length=N_test, trainer=Trainer(epochs=40,learning_rate=0.0001),
    use_feat_dynamic_real=True,
    batch_size=16,
    num_layers=3,
    num_cells=120
)

predictor = estimator.train(ds_train,ds_val)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.01it/s, epoch=1/40, avg_epoch_loss=11.9]
1it [00:00, 20.06it/s, epoch=1/40, validation_avg_epoch_loss=11.2]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.31it/s, epoch=2/40, avg_epoch_loss=11.2]
1it [00:00, 19.92it/s, epoch=2/40, validation_avg_epoch_loss=11]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 10.84it/s, epoch=3/40, avg_epoch_loss=11]
1it [00:00, 21.31it/s, epoch=3/40, validation_avg_epoch_loss=11.2]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.33it/s, epoch=4/40, av

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 10.98it/s, epoch=31/40, avg_epoch_loss=10.2]
1it [00:00, 17.82it/s, epoch=31/40, validation_avg_epoch_loss=10.4]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.08it/s, epoch=32/40, avg_epoch_loss=10.3]
1it [00:00, 20.43it/s, epoch=32/40, validation_avg_epoch_loss=10.7]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.25it/s, epoch=33/40, avg_epoch_loss=10.2]
1it [00:00, 19.40it/s, epoch=33/40, validation_avg_epoch_loss=10.5]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 10.95it/s, epoch=34/4

# Cross-validation

In [72]:
v_pred = []
v_baseline = []
for reduction in range(30,int(len(df)/2),30):
    ds = PandasDataset(df.iloc[:-reduction], target="Consommation (MW)",feat_dynamic_real=exo_feature)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=ds,#dataset.test,  # test dataset
        predictor=predictor,  # predictor, réglé pour predire les N_test derniers jours du dataframe
        num_samples=100,  # number of sample paths we want for evaluation, is useful for confidence interval
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)

    # first entry of the time series list
    ts_entry = tss[0]

    forecast_entry = forecasts[0]

    y_baseline = np.ones(N_test)*df['Consommation (MW)'].iloc[-reduction-N_test-1]
    y_true = df['Consommation (MW)'].iloc[-reduction-N_test:-reduction]
    y_pred = forecasts[0].median

    mape_pred = mean_absolute_percentage_error(y_true, y_pred)
    mape_baseline = mean_absolute_percentage_error(y_true, y_baseline)
    v_pred = np.hstack((v_pred,np.array([mape_pred])))
    v_baseline = np.hstack((v_baseline,np.array([mape_baseline])))
    
print(f'mean_mape(y_pred) = {v_pred.mean()}')
print(f'mean_mape(y_baseline) = {v_baseline.mean()}')

mean_mape(y_pred) = 0.029338365374223907
mean_mape(y_baseline) = 0.06738786759686786


# Resultat

## Meilleur modèle

epoch = 30  
learning_rate = 0.0001  
batch_size = 16  
num_layers = 2  
num_cells = 110 

## MAPE 

RESULTAT :  
mape_pred &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = 0.027    
mape_baseline = 0.067 
